In [ ]:
import pickle
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from nm.research.global_ppm.py.data_manager import FactSetDataManager
from nm.research.global_ppm.py.roic_calculator import ROICCalculator
from nm.research.global_ppm.py.wacc_calculator import WACCCalculator, CountryRiskParams
from nm.research.global_ppm.utils.financial_metrics_utils import (
    plot_metric_distribution, plot_metric_histogram,
    plot_metric_time_series, calculate_metric_statistics,
    create_metrics_comparison_plot, create_time_series_comparison,
    create_correlation_heatmap
)
from nm.research.global_ppm.utils.country_code_manager import get_country_manager


In [ ]:
# データ読み込み
pkl_path = "/home/tmiyahara/repos/Neumann-Notebook/tmiyahara/202507/processed_data/processed_dataset.pkl"
with open(pkl_path, 'rb') as f:
    processed_data = pickle.load(f)

country_manager = get_country_manager()

raw_data = processed_data['raw_data']
pivot_tables = processed_data['pivot_tables']
entity_info = processed_data['entity_info'].copy()

# 分析用データの準備
# 連結ROICデータの準備
consol_data = raw_data['consol'].copy()
segment_data = raw_data['segment'].copy()
consol_data.columns = ['FTERM_2', 'FACTSET_ENTITY_ID', 'ROIC(運用ベース)']

# 企業情報とマージして国情報を追加
entity_country = entity_info[['FACTSET_ENTITY_ID', 'ISO_COUNTRY_FACT']].drop_duplicates()
consol_data = consol_data.merge(entity_country, on='FACTSET_ENTITY_ID', how='left')

# 国コードから国名へのマッピング
consol_data['COUNTRY_NAME'] = consol_data['ISO_COUNTRY_FACT'].apply(
    lambda x: country_manager.get_country_name(x) if pd.notna(x) else None
    )

# セグメントROICデータの準備
segment_roic_pivot = pivot_tables['Yseg_consol']
segment_data = segment_roic_pivot.stack().to_frame("SEG_ROIC(運用ベース)").reset_index()
segment_data.columns = ['FTERM_2', 'CODE_SEGMENT_ADJ', 'SEG_ROIC(運用ベース)']
# WACC列の設定
wacc_cols = ['WACC']


In [ ]:
# ROIC分布可視化
plot_metric_distribution(
    consol_data,
    metric_column='ROIC(運用ベース)',
    width=1200,
    height=600
)


plot_metric_distribution(
    segment_data,
    metric_column='SEG_ROIC(運用ベース)',
    width=1200,
    height=600
)

In [ ]:
# WACC分布可視化
wacc_pivot = pivot_tables[wacc_pivot_keys[0]]
wacc_data = wacc_pivot.stack().to_frame("WACC").reset_index()
wacc_data.columns = ['FTERM_2', 'FACTSET_ENTITY_ID', 'WACC']

plot_metric_distribution(
    wacc_data,
    metric_column='WACC',
    width=1200,
    height=600
)

In [ ]:
# ヒストグラム可視化
fig = plot_metric_histogram_plotly(
    consol_data,
    metric_column='ROIC(運用ベース)',
    bins=50,
    width=800,
    height=500
)
fig.show()

fig = plot_metric_histogram_plotly(
    consol_data,
    metric_column=wacc_cols[0],
    bins=50,
    width=800,
    height=500
)
fig.show()


In [ ]:
# 指標間比較（ROIC vs WACC）
# ROICとWACCデータをマージ
comparison_data = consol_data.merge(
    wacc_data[['FTERM_2', 'FACTSET_ENTITY_ID', 'WACC']], 
    on=['FTERM_2', 'FACTSET_ENTITY_ID'], 
    how='inner'
)

fig = create_metrics_comparison_plot(
    comparison_data,
    x_metric='WACC',
    y_metric='ROIC(運用ベース)',
    title='ROIC vs WACC 散布図',
    width=800,
    height=600
)

fig.show()


In [ ]:
# 時系列比較
time_series_data = {}
metric_columns = {}

consol_roic_pivot = pivot_tables['Y_consol']
roic_time_series = consol_roic_pivot.mean(axis=0).reset_index()
roic_time_series.columns = ['FTERM_2', 'ROIC']
time_series_data['連結ROIC'] = roic_time_series
metric_columns['連結ROIC'] = 'ROIC'

wacc_pivot = pivot_tables[wacc_pivot_keys[0]]
wacc_time_series = wacc_pivot.mean(axis=0).reset_index()
wacc_time_series.columns = ['FTERM_2', 'WACC']
time_series_data['WACC'] = wacc_time_series
metric_columns['WACC'] = 'WACC'

fig = create_time_series_comparison(
    time_series_data,
    metric_columns,
    title='ROIC & WACC 時系列トレンド',
    width=1200,
    height=600
)
fig.show()

In [ ]:
# 個別企業時系列
consol_pivot = pivot_tables['Y_consol']

entity_ids = consol_pivot.index[:3]

for entity_id in entity_ids:
    fig = plot_metric_time_series(
        pd.DataFrame(consol_pivot.loc[entity_id]).T,
        entity_id=entity_id,
        fsym_id=entity_id,
        metric_name='連結ROIC',
        width=900,
        height=400
    )
    fig.show()


In [ ]:
# 国別分析
# 国別ボックスプロット
data_clean = consol_data.dropna(subset=['ROIC(運用ベース)', 'COUNTRY_NAME'])

country_counts = data_clean['COUNTRY_NAME'].value_counts()
top_countries = country_counts.head(10).index.tolist()
data_filtered = data_clean[data_clean['COUNTRY_NAME'].isin(top_countries)]

fig = px.box(
    data_filtered,
    x='COUNTRY_NAME',
    y='ROIC(運用ベース)',
    title='連結ROIC - 国別分布',
    height=600
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()

# 国別時系列トレンド
data_clean_time = data_clean.dropna(subset=['FTERM_2'])
top_countries_time = country_counts.head(6).index.tolist()
data_filtered_time = data_clean_time[data_clean_time['COUNTRY_NAME'].isin(top_countries_time)]

monthly_medians = data_filtered_time.groupby(['FTERM_2', 'COUNTRY_NAME'])['ROIC(運用ベース)'].median().reset_index()

fig = px.line(
    monthly_medians,
    x='FTERM_2',
    y='ROIC(運用ベース)',
    color='COUNTRY_NAME',
    title='連結ROIC - 国別時系列トレンド',
    height=600
)
fig.show()


In [ ]:
# 相関分析
if len(time_series_data) >= 2:
    merged_data = None
    for key, data in time_series_data.items():
        col = metric_columns[key]
        time_series = data[['FTERM_2', col]].rename(columns={col: key})
        
        if merged_data is None:
            merged_data = time_series
        else:
            merged_data = pd.merge(merged_data, time_series, on='FTERM_2', how='inner')
    
    if merged_data is not None and len(merged_data) > 2:
        metrics = [col for col in merged_data.columns if col != 'FTERM_2']
        if len(metrics) >= 2:
            fig = create_correlation_heatmap(
                merged_data,
                metrics,
                title="財務指標相関マトリックス",
                width=600,
                height=500
            )
            fig.show()


In [ ]:
# 基本統計量
statistics = {}

roic_stats = calculate_metric_statistics(consol_data, ['ROIC(運用ベース)'])
statistics.update(roic_stats)

wacc_stats = calculate_metric_statistics(wacc_data, ['WACC'])
statistics.update(wacc_stats)

seg_roic_stats = calculate_metric_statistics(segment_data, ['SEG_ROIC(運用ベース)'])
statistics.update(seg_roic_stats)

stats_df = pd.DataFrame(statistics).round(4)
stats_df